In [1]:
import arepo
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import matplotlib as mpl

from numba import njit

from joblib import Parallel, delayed

In [2]:
basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas/'

In [64]:
sn_MW = arepo.Snapshot(basepath + '/ics/MW3iso_fg0.7_MHG0.25_RC9/lvl4/MW_ICs.dat')
sn_GSE = arepo.Snapshot(basepath + '/ics//GSE5iso_fg0.7/lvl4/GSE_ICs.dat')

In [8]:
def rt_eq2(R_GSE=80.):
    M_GSE = 0.0
    M_MW = 0.0
    
    for i in range(6):
        M_MW += sn_MW.MassTable[i] * sn_MW.NumPart_Total[i]
        M_GSE += sn_GSE.MassTable[i] * sn_GSE.NumPart_Total[i]
    
    rt = R_GSE * (M_GSE / (3*M_MW))**(1./3.)
    
    return rt

In [74]:
def Menc(Rcut, sn, center=np.array([0., 0., 0.])):
    ans = 0.0
    for i in range(6):
        if sn.NumPart_Total[i] > 0:
            part = getattr(sn, 'part'+str(i))
            r = np.linalg.norm(part.pos-center, axis=1)
            key = np.where(r < Rcut)[0]
            if sn.MassTable[i] > 0.0:
                ans += sn.MassTable[i] * len(key)
            else:
                ans += np.sum(part.mass[key])
    
    return ans
            
def rt_eq6(R_GSE=80.):
    M_MW = Menc(R_GSE, sn_MW)
    # print(M_MW)
    
    # print(M_MW)
    # for a hernquist halo (the only material at R_Sgr)
    rt_guess = rt_eq2(R_GSE)

    # print(R_GSE)
    # print(Menc(rt_guess, sn_GSE))
    
    for _ in range(10):
        rt = R_GSE * ((Menc(rt_guess, sn_GSE)/M_MW))**(1./3.)
        rt_guess = rt
    
    return rt

In [77]:
print('eq2:, ', rt_eq2(25))
print('eq6:, ', rt_eq6(25))

eq2:,  14.895604923920427
eq6:,  18.930778216615444


In [76]:
rt_eq6(25)

18.930778216615444

In [57]:
sn_MW.MassTable[1] * sn_MW.NumPart_Total[1]

32.5677794107439

In [79]:
sn_GSE.MassTable[0] * sn_GSE.NumPart_Total[0]

0.11687058729418208